In [135]:
#importing libraries
import pandas as pd
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import numpy as np

In [3]:
df=pd.read_excel('Toronto.xlsx')
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
k=df[df['Neighborhood']=='Not assigned']
k[k['Borough']!='Not assigned']

,Postal Code,Borough,Neighborhood


## Before Cleaning Shape of a Data Frame

In [4]:
df.shape

(180, 3)

### Only processing the cells that have an assigned borough. Ignoring cells with a borough that is Not assigned

In [5]:
df_sorted=df[df['Borough']!='Not assigned'].reset_index(drop=True)
df_sorted.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Checking whether the cell has a borough but a Not assigned neighborhood

In [30]:
df_not=df_sorted[df_sorted['Neighborhood']=='Not assigned'] 
df_pending=df_not[df_not['Borough']!='Not assigned']
df_pending.count()

Postal Code     0
Borough         0
Neighborhood    0
dtype: int64

In [31]:
df_sorted.shape

(103, 3)

In [35]:
df_sorted.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
len(df_sorted['Borough'].unique())

10

In [10]:
len(df_sorted['Postal Code'].unique())

103

In [20]:
Cord_df=pd.read_csv('Geospatial_Coordinates.csv')
Cord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Joining Co-ordinates file to get latitudes and longitutdes for Dataframe df_sorted

In [26]:
df_sorted=df_sorted.join(Cord_df.set_index('Postal Code'),on='Postal Code')

In [33]:
df_sorted.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [34]:
df_sorted['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [35]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_sorted['Borough'].unique()),
        df_sorted.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Data Frame that contain details corresponding to North York

In [36]:
Ny_df=df_sorted[df_sorted['Borough']=='North York'].reset_index(drop=True)
Ny_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [38]:
Ny_df.shape

(24, 5)

In [39]:
CLIENT_ID = 'VFVS1F20ZBOXXTP3RBKNI1JLS3CJTWG4RGAWHPDY11FFCT33' # your Foursquare ID
CLIENT_SECRET = '0WYFLCNHHMBJ5H5A5VIGNWNCZQGN2B3OZXHAKYZ3YQG05K1I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VFVS1F20ZBOXXTP3RBKNI1JLS3CJTWG4RGAWHPDY11FFCT33
CLIENT_SECRET:0WYFLCNHHMBJ5H5A5VIGNWNCZQGN2B3OZXHAKYZ3YQG05K1I


### Function to return the venue details corresponding to each Neighborhood

In [95]:
def getnearbyvenues(name,lat,lng,radius):
    print(name)
    Ny_url="https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,20)
    Ny_venue=requests.get(Ny_url).json()['response']['groups'][0]['items']
    Ny_venues=json_normalize(Ny_venue)
    Category_list=[]
    try:
        for i in range(len(Ny_venues['venue.categories'])):
            Category_list.append(Ny_venues.loc[i,'venue.categories'][0]['name'])
    except:
        return None
    Ny_sorted_df=pd.DataFrame({'Neighborhood_name':name,'Neighborhood_lat':lat,'Neighborhood_lng':lng,'Name':Ny_venues['venue.name'],'Category':Category_list,'Latitude':Ny_venues['venue.location.lat'],'Longitude':Ny_venues['venue.location.lng']})
    l.append(Ny_sorted_df)
    return l        

### Verifying details of York Mills, Silver Hills

In [91]:
ysmh_url="https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,43.757490,-79.374714,500,20)
ysmh_venue=requests.get(Ny_url).json()
ysmh_v=Ny_venue['response']['groups'][0]['items']
ysmh_venues=json_normalize(Ny_v)

### It does'nt contain any items i.e., Categories

In [92]:
ysmh_venue['response']['groups'][0]

{'type': 'Recommended Places', 'name': 'recommended', 'items': []}

In [96]:
l=[]
for name,lat,lng in zip(Ny_df['Neighborhood'],Ny_df['Latitude'],Ny_df['Longitude']):
    test=getnearbyvenues(name,lat,lng,500)

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [103]:
North_york=pd.DataFrame()

In [104]:
for i in range(len(l)):
    North_york=North_york.append(l[i],ignore_index=True) 

In [105]:
North_york.head()

,Neighborhood_name,Neighborhood_lat,Neighborhood_lng,Name,Category,Latitude,Longitude
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park,43.751976,-79.332140
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,Bus Stop,43.752672,-79.326351
2,Parkwoods,43.753259,-79.329656,Variety Store,Food & Drink Shop,43.751974,-79.333114
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
4,Victoria Village,43.725882,-79.315572,Portugril,Portuguese Restaurant,43.725819,-79.312785


In [106]:
North_york.shape

(177, 7)

In [107]:
North_york.groupby('Neighborhood_name').count()

,Neighborhood_lat,Neighborhood_lng,Name,Category,Latitude,Longitude
Neighborhood_name,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Don Mills,25,25,25,25,25,25
Downsview,15,15,15,15,15,15
"Fairview, Henry Farm, Oriole",20,20,20,20,20,20
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,2,2,2,2,2,2


#### Getting Duplicate values w.r.t Categories

In [109]:
Ny_onehot=pd.get_dummies(North_york['Category'])
Ny_onehot['Neighborhood']=North_york['Neighborhood_name']
filtered_columns=[Ny_onehot.columns[-1]]+list(Ny_onehot.columns[:-1])
Ny_onehot=Ny_onehot[filtered_columns]

In [110]:
Ny_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vietnamese Restaurant
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
Ny_onehot.shape

(177, 88)

#### Calculating frequency of each category for a Neighborhood

In [114]:
Ny_grouped=Ny_onehot.groupby('Neighborhood').mean().reset_index()

In [115]:
Ny_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vietnamese Restaurant
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.00,0.00,0.0,0.00,0.000000,0.0,0.100000,...,0.00,0.0,0.05,0.05,0.0,0.00,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.00,0.00,0.0,0.00,0.000000,0.0,0.250000,...,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.05,0.00,0.0,0.00,0.000000,0.0,0.000000,...,0.00,0.0,0.00,0.05,0.0,0.05,0.0,0.0,0.0,0.0
3,Don Mills,0.0,0.000000,0.00,0.04,0.0,0.04,0.040000,0.0,0.000000,...,0.04,0.0,0.04,0.00,0.0,0.00,0.0,0.0,0.0,0.0
4,Downsview,0.0,0.066667,0.00,0.00,0.0,0.00,0.066667,0.0,0.066667,...,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [116]:
Ny_grouped.shape

(19, 88)

#### Function to return most common venues

In [117]:
def return_most_common_venues(row,num_top_venues):
    t=row.iloc[1:]
    row_categories_sorted=t.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [118]:
num_top_venues=10
indicator=['st','nd','rd']
columns=['Neighborhood']
for i in range(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1,indicator[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))
neighborhood_venues_sorted=pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood']=Ny_grouped['Neighborhood']
for i in range(len(Ny_grouped['Neighborhood'])):
    neighborhood_venues_sorted.iloc[i,1:]=return_most_common_venues(Ny_grouped.iloc[i,:],num_top_venues)
neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Mobile Phone Shop,Diner,Pizza Place,Ice Cream Shop,Deli / Bodega,Restaurant,Sandwich Place,Pharmacy
1,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Greek Restaurant,Pub,Comfort Food Restaurant,Pharmacy,Grocery Store,Restaurant,Juice Bar
3,Don Mills,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café,Italian Restaurant
4,Downsview,Park,Grocery Store,Snack Place,Gym / Fitness Center,Discount Store,Liquor Store,Business Service,Baseball Field,Shopping Mall,Food Truck


#### Clustering the Data

In [141]:
kclusters=5
Ny_grouped_clustering=Ny_grouped.drop('Neighborhood',axis=1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(Ny_grouped_clustering)
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 2, 1, 2, 4])

In [144]:
#neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Ny_merged=Ny_df
Ny_merged=Ny_merged.join(neighborhood_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
Ny_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Food & Drink Shop,Bus Stop,Park,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Vietnamese Restaurant,Steakhouse,Deli / Bodega
3,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café,Italian Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,2.0,Italian Restaurant,Japanese Restaurant,Pizza Place,Bakery,Pub,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


#### Droping the row 12 because it contains value NaN

In [145]:
Ny_merged=Ny_merged.drop([12],axis=0)

#### Plotting the clusters Map

In [158]:
map_clusters=folium.Map(location=[43.753259,-79.329656],zoom_stat=20)
x=np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat, lon, poi, cluster in zip(Ny_merged['Latitude'], Ny_merged['Longitude'], Ny_merged['Neighborhood'], Ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining Clusters

##### Cluster 1

In [150]:
Ny_merged.loc[Ny_merged['Cluster Labels'] == 0, Ny_merged.columns[[1] + list(range(5, Ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Food & Drink Shop,Bus Stop,Park,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
22,North York,0,Park,Convenience Store,Gift Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Deli / Bodega,Department Store


#### Cluster 2

In [153]:
Ny_merged.loc[Ny_merged['Cluster Labels'] == 1, Ny_merged.columns[[1] + list(range(5, Ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,North York,1,Clothing Store,Accessories Store,Furniture / Home Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Vietnamese Restaurant,Steakhouse,Deli / Bodega
3,North York,1,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café,Italian Restaurant
5,North York,1,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café,Italian Restaurant
6,North York,1,Golf Course,Pool,Mediterranean Restaurant,Athletics & Sports,Dog Run,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
7,North York,1,Coffee Shop,Bank,Mobile Phone Shop,Diner,Pizza Place,Ice Cream Shop,Deli / Bodega,Restaurant,Sandwich Place,Pharmacy
8,North York,1,Juice Bar,Clothing Store,Coffee Shop,Movie Theater,Shopping Mall,Chocolate Shop,Burger Joint,Pharmacy,Department Store,Salon / Barbershop
9,North York,1,Vietnamese Restaurant,Bar,Massage Studio,Miscellaneous Shop,Coffee Shop,Caribbean Restaurant,Furniture / Home Store,Athletics & Sports,American Restaurant,Construction & Landscaping
10,North York,1,Chinese Restaurant,Japanese Restaurant,Café,Bank,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store
11,North York,1,Park,Grocery Store,Snack Place,Gym / Fitness Center,Discount Store,Liquor Store,Business Service,Baseball Field,Shopping Mall,Food Truck


#### Cluster 3

In [154]:
Ny_merged.loc[Ny_merged['Cluster Labels'] == 2, Ny_merged.columns[[1] + list(range(5, Ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,North York,2,Italian Restaurant,Japanese Restaurant,Pizza Place,Bakery,Pub,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
15,North York,2,Pizza Place,Italian Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega


#### Cluster 4

In [155]:
Ny_merged.loc[Ny_merged['Cluster Labels'] == 3, Ny_merged.columns[[1] + list(range(5, Ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,North York,3,Piano Bar,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store


#### CLuster 5

In [156]:
Ny_merged.loc[Ny_merged['Cluster Labels'] == 4, Ny_merged.columns[[1] + list(range(5, Ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,4,Baseball Field,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant
